# Investment Research Agent - Multi-Agent System

## Project Overview
This notebook implements an autonomous Investment Research Agent that demonstrates:

## Architecture
- **Multi-Agent System**: Coordinator, Specialist Agents (News, Technical, Fundamental)
- **Memory System**: FAISS vector database for persistent learning
- **Data Sources**: Yahoo Finance, NewsAPI, FRED, Alpha Vantage
- **Interface**: Gradio web interface for user interaction

## 1. Environment Setup and Dependencies

In [16]:
# Install required packages
!pip install langchain langchain-openai langchain-community yfinance pandas numpy matplotlib seaborn plotly gradio faiss-cpu python-dotenv requests fredapi newsapi-python chromadb

In [17]:
# Import required libraries
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import warnings
import logging
import time
from typing import Dict, List, Any, Optional
import gradio as gr

# LangChain imports
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import BaseTool, tool
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import BaseMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.docstore.document import Document

# Data source imports
import yfinance as yf
from newsapi import NewsApiClient
from fredapi import Fred
import requests

# Environment variables
from dotenv import load_dotenv
load_dotenv()

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)

In [18]:
# Configuration from environment variables
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_GPT_DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_GPT_DEPLOYMENT_NAME')
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
AZURE_OPENAI_GPT_API_VERSION = os.getenv('AZURE_OPENAI_GPT_API_VERSION', '2024-02-15-preview')
AZURE_OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION', '2024-02-15-preview')
AZURE_OPENAI_EMBEDDING_API_VERSION = os.getenv('AZURE_OPENAI_EMBEDDING_API_VERSION', '2024-02-15-preview')

ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')
NEWS_API_KEY = os.getenv('NEWSAPI_KEY')
FRED_API_KEY = os.getenv('FRED_API_KEY')

# Initialize Azure OpenAI
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_GPT_DEPLOYMENT_NAME,
    openai_api_version=AZURE_OPENAI_GPT_API_VERSION,
    openai_api_key=AZURE_OPENAI_API_KEY
)

# Initialize embeddings for vector database
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME,
    openai_api_version=AZURE_OPENAI_EMBEDDING_API_VERSION,
    openai_api_key=AZURE_OPENAI_API_KEY
)

print("Environment setup completed successfully!")

Environment setup completed successfully!


In [19]:
class PromptConfiguration:
    """Central configuration class for all prompts used in the investment research system"""
    
    @staticmethod
    def get_planning_prompt(role: str, task: str) -> str:
        return f"""
        As an {role}, create a detailed research plan for: {task}
        
        Consider these aspects:
        1. Data gathering (price data, news, economic indicators, fundamentals)
        2. Analysis techniques (technical, fundamental, sentiment)
        3. Risk assessment
        4. Market context evaluation
        
        Return a numbered list of specific, actionable research steps.
        """
    
    @staticmethod
    def get_reflection_prompt(analysis_result: str) -> str:
        return f"""
        Please evaluate this investment analysis for quality and completeness:
        
        Analysis: {analysis_result}
        
        Provide a structured evaluation covering:
        1. Completeness (1-10): Are all key aspects covered?
        2. Data Quality (1-10): Is the data comprehensive and current?
        3. Logic (1-10): Is the reasoning sound and well-structured?
        4. Actionability (1-10): Are the conclusions practical and specific?
        5. Risk Assessment (1-10): Are risks properly identified and evaluated?
        
        Also provide:
        - Overall Score (1-10)
        - Key Strengths (2-3 points)
        - Areas for Improvement (2-3 points)
        - Specific Recommendations for enhancement
        
        Format as JSON with these exact keys: completeness, data_quality, logic, actionability, risk_assessment, overall_score, strengths, improvements, recommendations
        """
    
    @staticmethod
    def get_news_classification_prompt(title: str, description: str) -> str:
        return f"""
        Classify this news article:
        Title: {title}
        Description: {description}
        
        Provide classification in JSON format:
        {{
            "category": "earnings|product|market|regulation|management|merger|other",
            "sentiment": "positive|negative|neutral",
            "importance": "high|medium|low",
            "reasoning": "brief explanation"
        }}
        """
    
    @staticmethod
    def get_insights_extraction_prompt(classified_articles: str) -> str:
        return f"""
        Extract key insights from these classified news articles:
        
        {classified_articles}
        
        Provide insights in JSON format:
        {{
            "key_themes": ["list of main themes"],
            "sentiment_distribution": {{"positive": 0, "negative": 0, "neutral": 0}},
            "high_importance_items": ["list of high importance findings"],
            "potential_catalysts": ["events that could drive stock price"],
            "risk_factors": ["identified risks or concerns"]
        }}
        """
    
    @staticmethod
    def get_news_summarization_prompt(insights: str, symbol: str) -> str:
        return f"""
        Create a comprehensive news analysis summary for {symbol} based on these insights:
        
        {insights}
        
        Provide a professional investment-focused summary covering:
        1. Executive Summary (2-3 sentences)
        2. Key Developments and Themes
        3. Sentiment Analysis
        4. Potential Stock Price Catalysts
        5. Risk Factors to Monitor
        6. Investment Implications
        
        Keep the summary concise but comprehensive, suitable for investment decision-making.
        """
    
    @staticmethod
    def get_technical_analysis_prompt(symbol: str, stock_data: str) -> str:
        return f"""
        As a Technical Analysis Specialist, analyze the following stock data for {symbol}:
        
        Stock Data: {stock_data}
        
        Provide a comprehensive technical analysis covering:
        1. Price Trend Analysis (short-term and medium-term trends)
        2. Support and Resistance Levels
        3. Volume Analysis
        4. Key Technical Indicators (if calculable from available data)
        5. Chart Patterns (if identifiable)
        6. Technical Price Targets
        7. Risk Levels and Stop-Loss Recommendations
        
        Conclude with:
        - Technical Rating: Buy/Hold/Sell
        - Confidence Level: High/Medium/Low
        - Key Technical Risks
        - Next Important Price Levels to Watch
        """
    
    @staticmethod
    def get_fundamental_analysis_prompt(symbol: str, stock_data: str, alpha_overview: str) -> str:
        return f"""
        As a Fundamental Analysis Specialist, analyze {symbol} using this data:
        
        Basic Stock Data: {stock_data}
        Company Overview: {alpha_overview}
        
        Provide comprehensive fundamental analysis covering:
        1. Company Business Model and Competitive Position
        2. Financial Health Assessment
        3. Valuation Analysis (P/E, PEG, other relevant ratios)
        4. Growth Prospects and Market Opportunities
        5. Management Quality and Corporate Governance
        6. Industry and Sector Analysis
        7. Competitive Advantages and Moats
        
        Conclude with:
        - Fundamental Rating: Strong Buy/Buy/Hold/Sell/Strong Sell
        - Fair Value Estimate (if possible)
        - Key Fundamental Risks
        - Catalysts for Value Realization
        """
    
    @staticmethod  
    def get_sentiment_analysis_prompt(symbol: str, news_analysis: str) -> str:
        return f"""
        As a News and Sentiment Analysis Specialist, provide additional insights on this news analysis for {symbol}:
        
        {news_analysis}
        
        Focus on:
        1. Market Sentiment Implications
        2. News Flow Impact on Stock Price
        3. Institutional vs Retail Sentiment
        4. Social Media and Public Perception Trends
        5. News-Based Trading Opportunities
        6. Event-Driven Catalysts
        
        Conclude with:
        - Sentiment Rating: Very Positive/Positive/Neutral/Negative/Very Negative
        - News Impact Assessment: High/Medium/Low
        - Recommended Action Based on News Flow
        """
    
    @staticmethod
    def get_routing_prompt(request: str, symbol: str) -> str:
        return f"""
        Analyze this investment research request and determine which specialists should handle it:
        
        Request: {request}
        Symbol: {symbol}
        
        Available specialists:
        - technical: Technical analysis, chart patterns, price trends
        - fundamental: Company financials, valuation, business analysis
        - news: News analysis, sentiment, market events
        
        Return JSON format:
        {{
            "specialists_needed": ["list of specialist types needed"],
            "priority_order": ["ordered list by importance"],
            "reasoning": "why these specialists were chosen"
        }}
        """
    
    @staticmethod
    def get_analysis_generation_prompt(symbol: str, specialist_analyses: str) -> str:
        return f"""
        As a Senior Investment Analyst, create a comprehensive investment analysis for {symbol} 
        using these specialist reports:
        
        {specialist_analyses}
        
        Create a structured investment analysis with:
        1. Executive Summary
        2. Investment Thesis
        3. Key Strengths and Opportunities
        4. Risks and Concerns
        5. Financial Analysis Summary
        6. Technical Analysis Summary
        7. Market Sentiment Analysis
        8. Price Target and Recommendation
        9. Risk-Adjusted Return Expectations
        10. Conclusion and Action Items
        
        Make it comprehensive but concise, suitable for investment decision-making.
        """
    
    @staticmethod
    def get_evaluation_prompt(analysis: str, symbol: str) -> str:
        return f"""
        As an Investment Analysis Quality Evaluator, assess this investment analysis for {symbol}:
        
        Analysis:
        {analysis}
        
        Evaluate on these criteria (1-10 scale):
        1. Completeness: Are all key investment aspects covered?
        2. Data Integration: How well are different data sources synthesized?
        3. Risk Assessment: Is risk analysis comprehensive and realistic?
        4. Actionability: Are recommendations specific and implementable?
        5. Logic and Reasoning: Is the analysis logical and well-structured?
        6. Market Context: Is broader market context considered?
        7. Clarity: Is the analysis clear and professional?
        
        Provide feedback in JSON format:
        {{
            "scores": {{
                "completeness": X,
                "data_integration": X,
                "risk_assessment": X,
                "actionability": X,
                "logic_reasoning": X,
                "market_context": X,
                "clarity": X
            }},
            "overall_score": X,
            "grade": "A|B|C|D|F",
            "strengths": ["list of key strengths"],
            "weaknesses": ["list of areas needing improvement"],
            "specific_improvements": ["detailed suggestions for enhancement"],
            "missing_elements": ["what's missing from the analysis"]
        }}
        """
    
    @staticmethod
    def get_refinement_prompt(original_analysis: str, evaluation: str, symbol: str) -> str:
        return f"""
        Improve this investment analysis for {symbol} based on the evaluation feedback:
        
        Original Analysis:
        {original_analysis}
        
        Evaluation Feedback:
        {evaluation}
        
        Create an improved version that addresses these issues:
        1. Fix identified weaknesses
        2. Add missing elements
        3. Implement specific improvements
        4. Enhance overall quality and completeness
        5. Maintain professional investment analysis standards
        
        Focus particularly on areas that scored below 7/10 in the evaluation.
        """

print("Prompt configuration class created!")

Prompt configuration class created!


## 2. Data Source Tools and Integrations

In [20]:
# Yahoo Finance Tool
@tool
def get_stock_data(symbol: str, period: str = "1y") -> str:
    """Get stock price data and basic financial information from Yahoo Finance.
    
    Args:
        symbol: Stock symbol (e.g., 'AAPL', 'MSFT')
        period: Time period ('1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max')
    """
    try:
        stock = yf.Ticker(symbol)
        hist = stock.history(period=period)
        info = stock.info
        
        current_price = hist['Close'].iloc[-1]
        price_change = hist['Close'].iloc[-1] - hist['Close'].iloc[-2]
        price_change_pct = (price_change / hist['Close'].iloc[-2]) * 100
        
        result = {
            'symbol': symbol,
            'current_price': round(current_price, 2),
            'price_change': round(price_change, 2),
            'price_change_pct': round(price_change_pct, 2),
            'volume': hist['Volume'].iloc[-1],
            'market_cap': info.get('marketCap', 'N/A'),
            'pe_ratio': info.get('trailingPE', 'N/A'),
            'company_name': info.get('longName', 'N/A'),
            'sector': info.get('sector', 'N/A'),
            'industry': info.get('industry', 'N/A')
        }
        
        return json.dumps(result, indent=2)
    
    except Exception as e:
        return f"Error fetching stock data for {symbol}: {str(e)}"

# News API Tool
@tool
def get_stock_news(symbol: str, days: int = 7) -> str:
    """Get recent news articles related to a stock symbol.
    
    Args:
        symbol: Stock symbol to search news for
        days: Number of days to look back for news
    """
    try:
        newsapi = NewsApiClient(api_key=NEWS_API_KEY)
        
        from_date = (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
        
        articles = newsapi.get_everything(
            q=symbol,
            language='en',
            sort_by='relevancy',
            from_param=from_date,
            page_size=10
        )
        
        news_items = []
        for article in articles['articles'][:5]:  # Top 5 articles
            news_items.append({
                'title': article['title'],
                'description': article['description'],
                'source': article['source']['name'],
                'published_at': article['publishedAt'],
                'url': article['url']
            })
        
        return json.dumps(news_items, indent=2)
    
    except Exception as e:
        return f"Error fetching news for {symbol}: {str(e)}"

# FRED Economic Data Tool
@tool
def get_economic_data(series_id: str = "GDP") -> str:
    """Get economic data from FRED (Federal Reserve Economic Data).
    
    Args:
        series_id: FRED series ID (e.g., 'GDP', 'UNRATE', 'FEDFUNDS', 'CPIAUCSL')
    """
    try:
        fred = Fred(api_key=FRED_API_KEY)
        data = fred.get_series(series_id, limit=12)  # Last 12 observations
        
        result = {
            'series_id': series_id,
            'latest_value': float(data.iloc[-1]),
            'latest_date': data.index[-1].strftime('%Y-%m-%d'),
            'previous_value': float(data.iloc[-2]),
            'change': float(data.iloc[-1] - data.iloc[-2]),
            'change_pct': float((data.iloc[-1] - data.iloc[-2]) / data.iloc[-2] * 100)
        }
        
        return json.dumps(result, indent=2)
    
    except Exception as e:
        return f"Error fetching economic data for {series_id}: {str(e)}"

# Alpha Vantage Tool
@tool
def get_alpha_vantage_data(symbol: str, function: str = "TIME_SERIES_DAILY") -> str:
    """Get financial data from Alpha Vantage API.
    
    Args:
        symbol: Stock symbol
        function: Alpha Vantage function (e.g., 'TIME_SERIES_DAILY', 'OVERVIEW', 'INCOME_STATEMENT')
    """
    try:
        base_url = "https://www.alphavantage.co/query"
        params = {
            'function': function,
            'symbol': symbol,
            'apikey': ALPHA_VANTAGE_API_KEY
        }
        
        response = requests.get(base_url, params=params)
        data = response.json()
        
        # Return a simplified version to avoid token limits
        if function == "OVERVIEW":
            overview = {
                'symbol': data.get('Symbol', 'N/A'),
                'market_cap': data.get('MarketCapitalization', 'N/A'),
                'pe_ratio': data.get('PERatio', 'N/A'),
                'peg_ratio': data.get('PEGRatio', 'N/A'),
                'dividend_yield': data.get('DividendYield', 'N/A'),
                'eps': data.get('EPS', 'N/A'),
                '52_week_high': data.get('52WeekHigh', 'N/A'),
                '52_week_low': data.get('52WeekLow', 'N/A')
            }
            return json.dumps(overview, indent=2)
        
        return json.dumps(data, indent=2)[:1000]  # Truncate to avoid token limits
    
    except Exception as e:
        return f"Error fetching Alpha Vantage data for {symbol}: {str(e)}"

print("Data source tools created successfully!")

Data source tools created successfully!


## 3. Vector Database and Memory System

In [21]:
class AgentMemory:
    """Persistent memory system using FAISS vector database."""
    
    def __init__(self, memory_path: str = "./database/agent_memory"):
        self.memory_path = memory_path
        self.vector_store = None
        self.initialize_memory()
    
    def initialize_memory(self):
        """Initialize or load existing memory."""
        try:
            # Try to load existing memory
            if os.path.exists(f"{self.memory_path}.faiss"):
                self.vector_store = FAISS.load_local(self.memory_path, embeddings)
                print("Loaded existing agent memory")
            else:
                # Create new memory with initial documents
                initial_docs = [
                    Document(page_content="Investment analysis requires considering multiple factors: technical indicators, fundamental analysis, market sentiment, and economic conditions.", 
                            metadata={"type": "general_knowledge", "timestamp": datetime.now().isoformat()})
                ]
                self.vector_store = FAISS.from_documents(initial_docs, embeddings)
                self.save_memory()
                print("Created new agent memory")
        except Exception as e:
            print(f"Error initializing memory: {e}")
            # Fallback: create minimal memory
            initial_docs = [
                Document(page_content="Fallback memory initialized", 
                        metadata={"type": "system", "timestamp": datetime.now().isoformat()})
            ]
            self.vector_store = FAISS.from_documents(initial_docs, embeddings)
    
    def add_memory(self, content: str, metadata: Dict[str, Any]):
        """Add new memory to the vector store."""
        try:
            doc = Document(page_content=content, metadata=metadata)
            self.vector_store.add_documents([doc])
            self.save_memory()
        except Exception as e:
            print(f"Error adding memory: {e}")
    
    def search_memory(self, query: str, k: int = 5) -> List[Document]:
        """Search for relevant memories."""
        try:
            return self.vector_store.similarity_search(query, k=k)
        except Exception as e:
            print(f"Error searching memory: {e}")
            return []
    
    def save_memory(self):
        """Save memory to disk."""
        try:
            self.vector_store.save_local(self.memory_path)
        except Exception as e:
            print(f"Error saving memory: {e}")

# Initialize global memory
agent_memory = AgentMemory()
print("Agent memory system initialized!")

INFO:httpx:HTTP Request: POST https://sasw-mgfv7ds1-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Created new agent memory
Agent memory system initialized!


## 4. Base Agent Class with Core Functions

In [22]:
class InvestmentResearchAgent:
    """Base Investment Research Agent with planning, tool usage, self-reflection, and learning capabilities."""
    
    def __init__(self, name: str, role: str, memory: AgentMemory):
        self.name = name
        self.role = role
        self.memory = memory
        self.llm = llm
        self.session_memory = ConversationBufferWindowMemory(
            k=10, 
            memory_key="chat_history", 
            return_messages=True
        )
        self.tools = [get_stock_data, get_stock_news, get_economic_data, get_alpha_vantage_data]
        self.execution_log = []
        
    def plan_research(self, task: str) -> List[str]:
        """Plan research steps for a given task."""
        planning_prompt = PromptConfiguration.get_planning_prompt(self.role, task)
        
        try:
            response = self.llm.invoke([HumanMessage(content=planning_prompt)])
            plan_text = response.content
            
            # Extract numbered steps
            steps = []
            for line in plan_text.split('\n'):
                line = line.strip()
                if line and (line[0].isdigit() or line.startswith('-') or line.startswith('*')):
                    steps.append(line)
            
            # Log the plan
            self.execution_log.append({
                'timestamp': datetime.now().isoformat(),
                'action': 'plan_created',
                'task': task,
                'plan': steps
            })
            
            return steps
        
        except Exception as e:
            print(f"Error in planning: {e}")
            return ["1. Gather basic stock data", "2. Analyze recent news", "3. Review economic context", "4. Synthesize findings"]
    
    def use_tool_dynamically(self, tool_name: str, **kwargs) -> str:
        """Use tools dynamically based on context."""
        tool_map = {
            'stock_data': get_stock_data,
            'news': get_stock_news,
            'economic': get_economic_data,
            'alpha_vantage': get_alpha_vantage_data
        }
        
        try:
            if tool_name in tool_map:
                result = tool_map[tool_name].invoke(kwargs)
                
                # Log tool usage
                self.execution_log.append({
                    'timestamp': datetime.now().isoformat(),
                    'action': 'tool_used',
                    'tool': tool_name,
                    'parameters': kwargs,
                    'success': True
                })
                
                return result
            else:
                return f"Tool '{tool_name}' not available"
        
        except Exception as e:
            self.execution_log.append({
                'timestamp': datetime.now().isoformat(),
                'action': 'tool_used',
                'tool': tool_name,
                'parameters': kwargs,
                'success': False,
                'error': str(e)
            })
            return f"Error using tool '{tool_name}': {str(e)}"
    
    def self_reflect(self, analysis_result: str) -> Dict[str, Any]:
        """Self-reflect on the quality of analysis output."""
        reflection_prompt = PromptConfiguration.get_reflection_prompt(analysis_result)
        
        try:
            response = self.llm.invoke([HumanMessage(content=reflection_prompt)])
            reflection_text = response.content
            
            # Try to parse as JSON, fallback to structured text parsing
            try:
                reflection_data = json.loads(reflection_text)
            except:
                # Fallback parsing
                reflection_data = {
                    'overall_score': 7,
                    'strengths': ["Analysis provided"],
                    'improvements': ["Could be more detailed"],
                    'recommendations': ["Gather more data points"]
                }
            
            # Log reflection
            self.execution_log.append({
                'timestamp': datetime.now().isoformat(),
                'action': 'self_reflection',
                'reflection': reflection_data
            })
            
            return reflection_data
        
        except Exception as e:
            print(f"Error in self-reflection: {e}")
            return {
                'overall_score': 5,
                'strengths': ["Attempt made"],
                'improvements': ["Technical issues encountered"],
                'recommendations': ["Retry analysis"]
            }
    
    def learn_from_experience(self, task: str, result: str, reflection: Dict[str, Any]):
        """Learn from the current analysis and store insights for future use."""
        try:
            # Create learning content
            learning_content = f"""
            Task: {task}
            Analysis Quality Score: {reflection.get('overall_score', 'N/A')}
            Key Insights: {', '.join(reflection.get('strengths', []))}
            Improvement Areas: {', '.join(reflection.get('improvements', []))}
            Recommendations: {', '.join(reflection.get('recommendations', []))}
            Execution Log: {len(self.execution_log)} actions taken
            """
            
            # Store in memory
            metadata = {
                'type': 'learning_experience',
                'agent': self.name,
                'task': task,
                'timestamp': datetime.now().isoformat(),
                'quality_score': reflection.get('overall_score', 0)
            }
            
            self.memory.add_memory(learning_content, metadata)
            
            print(f"{self.name} learned from experience (Score: {reflection.get('overall_score', 'N/A')})")
        
        except Exception as e:
            print(f"Error in learning: {e}")
    
    def retrieve_relevant_experience(self, task: str) -> List[str]:
        """Retrieve relevant past experiences for the current task."""
        try:
            memories = self.memory.search_memory(task, k=3)
            experiences = []
            
            for memory in memories:
                if memory.metadata.get('type') == 'learning_experience':
                    experiences.append(memory.page_content)
            
            return experiences
        
        except Exception as e:
            print(f"Error retrieving experience: {e}")
            return []

print("Base Investment Research Agent class created!")

Base Investment Research Agent class created!


## 5. Workflow Pattern 1: Prompt Chaining (News Processing Pipeline)

In [23]:
class NewsProcessingChain:
    """Implements Prompt Chaining: Ingest → Preprocess → Classify → Extract → Summarize"""
    
    def __init__(self, llm):
        self.llm = llm
    
    def ingest_news(self, symbol: str) -> List[Dict]:
        """Step 1: Ingest news data"""
        try:
            news_data = get_stock_news.invoke({"symbol": symbol, "days": 7})
            
            # Handle empty or None response
            if not news_data or news_data.strip() == "":
                print(f"No news data returned for {symbol}")
                return []
            
            # Check if response is an error message (string starting with "Error")
            if isinstance(news_data, str) and news_data.startswith("Error"):
                print(f"News API error: {news_data}")
                return []
            
            # Try to parse JSON
            try:
                parsed_data = json.loads(news_data)
                
                # Handle different response formats
                if isinstance(parsed_data, list):
                    return parsed_data
                elif isinstance(parsed_data, dict):
                    # If it's a dict with news items, extract them
                    if 'news' in parsed_data:
                        return parsed_data['news']
                    elif 'articles' in parsed_data:
                        return parsed_data['articles']
                    else:
                        # Return as single item list
                        return [parsed_data]
                else:
                    print(f"Unexpected news data format: {type(parsed_data)}")
                    return []
                    
            except json.JSONDecodeError as je:
                print(f"JSON parsing error: {je}")
                print(f"Raw response: {news_data[:200]}...")  # Show first 200 chars
                return []
                
        except Exception as e:
            print(f"Error ingesting news: {e}")
            return []
    
    def preprocess_news(self, news_articles: List[Dict]) -> List[Dict]:
        """Step 2: Preprocess and clean news articles"""
        processed_articles = []
        
        for article in news_articles:
            # Clean and structure the article
            processed_article = {
                'title': article.get('title', '').strip(),
                'description': article.get('description', '').strip(),
                'source': article.get('source', 'Unknown'),
                'published_at': article.get('published_at', ''),
                'url': article.get('url', ''),
                'combined_text': f"{article.get('title', '')} {article.get('description', '')}".strip()
            }
            
            # Only include articles with meaningful content
            if processed_article['combined_text'] and len(processed_article['combined_text']) > 20:
                processed_articles.append(processed_article)
        
        return processed_articles
    
    def classify_news(self, processed_articles: List[Dict]) -> List[Dict]:
        """Step 3: Classify news articles by type and sentiment"""
        classified_articles = []
        
        for article in processed_articles:
            classify_prompt = PromptConfiguration.get_news_classification_prompt(
                article['title'], 
                article['description']
            )
            
            try:
                response = self.llm.invoke([HumanMessage(content=classify_prompt)])
                classification_text = response.content
                
                # Try to parse JSON, fallback to default values
                try:
                    classification = json.loads(classification_text)
                except:
                    classification = {
                        "category": "other",
                        "sentiment": "neutral",
                        "importance": "medium",
                        "reasoning": "Classification parsing failed"
                    }
                
                article.update(classification)
                classified_articles.append(article)
            
            except Exception as e:
                print(f"Error classifying article: {e}")
                article.update({
                    "category": "other",
                    "sentiment": "neutral",
                    "importance": "medium",
                    "reasoning": "Error in classification"
                })
                classified_articles.append(article)
        
        return classified_articles
    
    def extract_insights(self, classified_articles: List[Dict]) -> Dict[str, Any]:
        """Step 4: Extract key insights from classified articles"""
        classified_articles_str = json.dumps(classified_articles, indent=2)[:2000]  # Truncate to avoid token limits
        insights_prompt = PromptConfiguration.get_insights_extraction_prompt(classified_articles_str)
        
        try:
            response = self.llm.invoke([HumanMessage(content=insights_prompt)])
            insights_text = response.content
            
            try:
                insights = json.loads(insights_text)
            except:
                # Fallback insights
                sentiment_counts = {"positive": 0, "negative": 0, "neutral": 0}
                for article in classified_articles:
                    sentiment = article.get('sentiment', 'neutral')
                    sentiment_counts[sentiment] += 1
                
                insights = {
                    "key_themes": ["General market activity"],
                    "sentiment_distribution": sentiment_counts,
                    "high_importance_items": [item['title'] for item in classified_articles if item.get('importance') == 'high'],
                    "potential_catalysts": ["Market developments"],
                    "risk_factors": ["Market volatility"]
                }
            
            return insights
        
        except Exception as e:
            print(f"Error extracting insights: {e}")
            return {
                "key_themes": ["Analysis error"],
                "sentiment_distribution": {"positive": 0, "negative": 0, "neutral": len(classified_articles)},
                "high_importance_items": [],
                "potential_catalysts": [],
                "risk_factors": ["Analysis uncertainty"]
            }
    
    def summarize_analysis(self, insights: Dict[str, Any], symbol: str) -> str:
        """Step 5: Summarize the complete news analysis"""
        insights_str = json.dumps(insights, indent=2)
        summarize_prompt = PromptConfiguration.get_news_summarization_prompt(insights_str, symbol)
        
        try:
            response = self.llm.invoke([HumanMessage(content=summarize_prompt)])
            return response.content
        
        except Exception as e:
            print(f"Error in summarization: {e}")
            return f"News analysis for {symbol} completed with {len(insights.get('key_themes', []))} key themes identified. Sentiment distribution shows mixed signals. Further analysis recommended."
    
    def process_news_chain(self, symbol: str) -> Dict[str, Any]:
        """Execute the complete news processing chain"""
        print(f"Starting news processing chain for {symbol}...")
        
        # Step 1: Ingest
        print("Step 1: Ingesting news...")
        raw_news = self.ingest_news(symbol)
        
        # Step 2: Preprocess
        print("Step 2: Preprocessing news...")
        processed_news = self.preprocess_news(raw_news)
        
        # Step 3: Classify
        print("Step 3: Classifying news...")
        classified_news = self.classify_news(processed_news)
        
        # Step 4: Extract
        print("Step 4: Extracting insights...")
        insights = self.extract_insights(classified_news)
        
        # Step 5: Summarize
        print("Step 5: Creating summary...")
        summary = self.summarize_analysis(insights, symbol)
        
        return {
            'symbol': symbol,
            'raw_articles_count': len(raw_news),
            'processed_articles_count': len(processed_news),
            'classified_articles': classified_news,
            'insights': insights,
            'summary': summary,
            'timestamp': datetime.now().isoformat()
        }

# Initialize news processing chain
news_chain = NewsProcessingChain(llm)
print("News Processing Chain (Prompt Chaining) created!")

News Processing Chain (Prompt Chaining) created!


## 6. Workflow Pattern 2: Routing (Specialist Agents)

In [24]:
class SpecialistAgent(InvestmentResearchAgent):
    """Base class for specialist agents"""
    
    def __init__(self, name: str, role: str, specialization: str, memory: AgentMemory):
        super().__init__(name, role, memory)
        self.specialization = specialization
    
    def analyze(self, data: Dict[str, Any], symbol: str) -> Dict[str, Any]:
        """Perform specialized analysis - to be implemented by subclasses"""
        raise NotImplementedError

class TechnicalAnalyst(SpecialistAgent):
    """Specialist agent for technical analysis"""
    
    def __init__(self, memory: AgentMemory):
        super().__init__("TechnicalAnalyst", "Technical Analysis Specialist", "technical_analysis", memory)
    
    def analyze(self, data: Dict[str, Any], symbol: str) -> Dict[str, Any]:
        """Perform technical analysis"""
        try:
            # Get stock data for technical analysis
            stock_data = self.use_tool_dynamically('stock_data', symbol=symbol, period='6mo')
            
            technical_prompt = PromptConfiguration.get_technical_analysis_prompt(symbol, stock_data)
            
            response = self.llm.invoke([HumanMessage(content=technical_prompt)])
            analysis = response.content
            
            return {
                'specialist': self.name,
                'analysis_type': 'technical',
                'symbol': symbol,
                'analysis': analysis,
                'timestamp': datetime.now().isoformat(),
                'data_used': ['stock_price_data', 'volume_data']
            }
        
        except Exception as e:
            return {
                'specialist': self.name,
                'analysis_type': 'technical',
                'symbol': symbol,
                'analysis': f"Technical analysis error: {str(e)}",
                'timestamp': datetime.now().isoformat(),
                'error': True
            }

class FundamentalAnalyst(SpecialistAgent):
    """Specialist agent for fundamental analysis"""
    
    def __init__(self, memory: AgentMemory):
        super().__init__("FundamentalAnalyst", "Fundamental Analysis Specialist", "fundamental_analysis", memory)
    
    def analyze(self, data: Dict[str, Any], symbol: str) -> Dict[str, Any]:
        """Perform fundamental analysis"""
        try:
            # Get fundamental data
            stock_data = self.use_tool_dynamically('stock_data', symbol=symbol)
            alpha_overview = self.use_tool_dynamically('alpha_vantage', symbol=symbol, function='OVERVIEW')
            
            fundamental_prompt = PromptConfiguration.get_fundamental_analysis_prompt(symbol, stock_data, alpha_overview)
            
            response = self.llm.invoke([HumanMessage(content=fundamental_prompt)])
            analysis = response.content
            
            return {
                'specialist': self.name,
                'analysis_type': 'fundamental',
                'symbol': symbol,
                'analysis': analysis,
                'timestamp': datetime.now().isoformat(),
                'data_used': ['company_financials', 'market_data', 'ratios']
            }
        
        except Exception as e:
            return {
                'specialist': self.name,
                'analysis_type': 'fundamental',
                'symbol': symbol,
                'analysis': f"Fundamental analysis error: {str(e)}",
                'timestamp': datetime.now().isoformat(),
                'error': True
            }

class NewsAnalyst(SpecialistAgent):
    """Specialist agent for news and sentiment analysis"""
    
    def __init__(self, memory: AgentMemory):
        super().__init__("NewsAnalyst", "News and Sentiment Analysis Specialist", "news_analysis", memory)
    
    def analyze(self, data: Dict[str, Any], symbol: str) -> Dict[str, Any]:
        """Perform news and sentiment analysis"""
        try:
            # Use the news processing chain
            news_analysis = news_chain.process_news_chain(symbol)
            
            news_analysis_str = json.dumps(news_analysis, indent=2)[:1500]
            sentiment_prompt = PromptConfiguration.get_sentiment_analysis_prompt(symbol, news_analysis_str)
            
            response = self.llm.invoke([HumanMessage(content=sentiment_prompt)])
            enhanced_analysis = response.content
            
            return {
                'specialist': self.name,
                'analysis_type': 'news_sentiment',
                'symbol': symbol,
                'analysis': enhanced_analysis,
                'raw_news_analysis': news_analysis,
                'timestamp': datetime.now().isoformat(),
                'data_used': ['news_articles', 'sentiment_data']
            }
        
        except Exception as e:
            return {
                'specialist': self.name,
                'analysis_type': 'news_sentiment',
                'symbol': symbol,
                'analysis': f"News analysis error: {str(e)}",
                'timestamp': datetime.now().isoformat(),
                'error': True
            }

class RoutingCoordinator:
    """Coordinates routing of analysis tasks to appropriate specialists"""
    
    def __init__(self, memory: AgentMemory):
        self.memory = memory
        self.specialists = {
            'technical': TechnicalAnalyst(memory),
            'fundamental': FundamentalAnalyst(memory),
            'news': NewsAnalyst(memory)
        }
        self.llm = llm
    
    def route_analysis(self, request: str, symbol: str) -> Dict[str, Any]:
        """Route analysis request to appropriate specialists"""
        routing_prompt = PromptConfiguration.get_routing_prompt(request, symbol)
        
        try:
            response = self.llm.invoke([HumanMessage(content=routing_prompt)])
            routing_text = response.content
            
            try:
                routing_decision = json.loads(routing_text)
            except:
                # Default routing - use all specialists
                routing_decision = {
                    "specialists_needed": ["technical", "fundamental", "news"],
                    "priority_order": ["fundamental", "technical", "news"],
                    "reasoning": "Default comprehensive analysis"
                }
            
            # Execute analysis with selected specialists
            results = {}
            for specialist_type in routing_decision.get('specialists_needed', []):
                if specialist_type in self.specialists:
                    print(f"Routing to {specialist_type} specialist...")
                    specialist_result = self.specialists[specialist_type].analyze({}, symbol)
                    results[specialist_type] = specialist_result
            
            return {
                'routing_decision': routing_decision,
                'specialist_analyses': results,
                'symbol': symbol,
                'request': request,
                'timestamp': datetime.now().isoformat()
            }
        
        except Exception as e:
            print(f"Error in routing: {e}")
            return {
                'error': f"Routing error: {str(e)}",
                'symbol': symbol,
                'timestamp': datetime.now().isoformat()
            }

# Initialize routing system
routing_coordinator = RoutingCoordinator(agent_memory)
print("Routing System (Specialist Agents) created!")

Routing System (Specialist Agents) created!


## 7. Workflow Pattern 3: Evaluator-Optimizer (Analysis Refinement Loop)

In [25]:
class EvaluatorOptimizer:
    """Implements Evaluator-Optimizer pattern: Generate → Evaluate → Refine"""
    
    def __init__(self, llm, memory: AgentMemory):
        self.llm = llm
        self.memory = memory
        self.max_iterations = 3
    
    def generate_initial_analysis(self, symbol: str, specialist_analyses: Dict[str, Any]) -> str:
        """Generate initial comprehensive analysis"""
        specialist_analyses_str = json.dumps(specialist_analyses, indent=2)[:3000]
        generate_prompt = PromptConfiguration.get_analysis_generation_prompt(symbol, specialist_analyses_str)
        
        try:
            response = self.llm.invoke([HumanMessage(content=generate_prompt)])
            return response.content
        except Exception as e:
            return f"Error generating initial analysis: {str(e)}"
    
    def evaluate_analysis_quality(self, analysis: str, symbol: str) -> Dict[str, Any]:
        """Evaluate the quality of the analysis using Azure OpenAI"""
        evaluation_prompt = PromptConfiguration.get_evaluation_prompt(analysis, symbol)
        
        try:
            response = self.llm.invoke([HumanMessage(content=evaluation_prompt)])
            evaluation_text = response.content
            
            try:
                evaluation = json.loads(evaluation_text)
            except:
                # Fallback evaluation
                evaluation = {
                    "scores": {
                        "completeness": 7,
                        "data_integration": 6,
                        "risk_assessment": 6,
                        "actionability": 7,
                        "logic_reasoning": 7,
                        "market_context": 6,
                        "clarity": 7
                    },
                    "overall_score": 6.5,
                    "grade": "B",
                    "strengths": ["Basic analysis provided"],
                    "weaknesses": ["Could be more detailed"],
                    "specific_improvements": ["Add more quantitative analysis"],
                    "missing_elements": ["Market comparisons"]
                }
            
            return evaluation
        
        except Exception as e:
            print(f"Error in evaluation: {e}")
            return {
                "overall_score": 5,
                "grade": "C",
                "strengths": ["Attempt made"],
                "weaknesses": ["Evaluation error"],
                "specific_improvements": ["Retry analysis"],
                "missing_elements": ["Complete analysis"]
            }
    
    def refine_analysis(self, original_analysis: str, evaluation: Dict[str, Any], symbol: str) -> str:
        """Refine analysis based on evaluation feedback"""
        evaluation_str = json.dumps(evaluation, indent=2)
        refinement_prompt = PromptConfiguration.get_refinement_prompt(original_analysis, evaluation_str, symbol)
        
        try:
            response = self.llm.invoke([HumanMessage(content=refinement_prompt)])
            return response.content
        except Exception as e:
            return f"Error in refinement: {str(e)}. Original analysis maintained."
    
    def optimize_analysis(self, symbol: str, specialist_analyses: Dict[str, Any]) -> Dict[str, Any]:
        """Execute the complete evaluator-optimizer loop"""
        print(f"Starting analysis optimization for {symbol}...")
        
        iterations = []
        current_analysis = self.generate_initial_analysis(symbol, specialist_analyses)
        
        for iteration in range(self.max_iterations):
            print(f"Optimization iteration {iteration + 1}/{self.max_iterations}")
            
            # Evaluate current analysis
            evaluation = self.evaluate_analysis_quality(current_analysis, symbol)
            
            iteration_data = {
                'iteration': iteration + 1,
                'analysis': current_analysis,
                'evaluation': evaluation,
                'timestamp': datetime.now().isoformat()
            }
            
            # Check if quality is acceptable (grade A or B, or score > 8)
            overall_score = evaluation.get('overall_score', 0)
            grade = evaluation.get('grade', 'F')
            
            if grade in ['A', 'B'] or overall_score >= 8.0:
                print(f"Analysis quality acceptable (Grade: {grade}, Score: {overall_score})")
                iteration_data['optimization_complete'] = True
                iterations.append(iteration_data)
                break
            
            # Refine analysis if quality is not acceptable
            if iteration < self.max_iterations - 1:  # Don't refine on last iteration
                print(f"Refining analysis (Grade: {grade}, Score: {overall_score})")
                current_analysis = self.refine_analysis(current_analysis, evaluation, symbol)
                iteration_data['refinement_applied'] = True
            else:
                print(f"Maximum iterations reached. Final grade: {grade}, Score: {overall_score}")
                iteration_data['max_iterations_reached'] = True
            
            iterations.append(iteration_data)
        
        # Store learning in memory
        final_evaluation = iterations[-1]['evaluation']
        learning_content = f"""
        Analysis Optimization for {symbol}:
        Iterations: {len(iterations)}
        Final Score: {final_evaluation.get('overall_score', 'N/A')}
        Final Grade: {final_evaluation.get('grade', 'N/A')}
        Key Learnings: {', '.join(final_evaluation.get('strengths', []))}
        """
        
        self.memory.add_memory(learning_content, {
            'type': 'optimization_learning',
            'symbol': symbol,
            'final_score': final_evaluation.get('overall_score', 0),
            'iterations': len(iterations),
            'timestamp': datetime.now().isoformat()
        })
        
        return {
            'symbol': symbol,
            'optimization_iterations': iterations,
            'final_analysis': current_analysis,
            'final_evaluation': final_evaluation,
            'improvement_achieved': len(iterations) > 1,
            'timestamp': datetime.now().isoformat()
        }

# Initialize evaluator-optimizer
evaluator_optimizer = EvaluatorOptimizer(llm, agent_memory)
print("Evaluator-Optimizer system created!")

Evaluator-Optimizer system created!


## 8. Main Investment Research Agent Coordinator

In [26]:
class MainInvestmentResearchAgent(InvestmentResearchAgent):
    """Main coordinator agent that orchestrates all workflows and patterns"""
    
    def __init__(self, memory: AgentMemory):
        super().__init__("MainCoordinator", "Investment Research Coordinator", memory)
        self.routing_coordinator = routing_coordinator
        self.evaluator_optimizer = evaluator_optimizer
        self.news_chain = news_chain
        
    def conduct_comprehensive_research(self, symbol: str, request: str = None) -> Dict[str, Any]:
        """
        Conduct comprehensive investment research using all three workflow patterns:
        1. Prompt Chaining (News Processing)
        2. Routing (Specialist Analysis) 
        3. Evaluator-Optimizer (Analysis Refinement)
        """
        print(f"\\nStarting comprehensive investment research for {symbol}")
        print("=" * 60)
        
        # Step 1: Planning
        if not request:
            request = f"Conduct comprehensive investment analysis for {symbol} including technical, fundamental, and sentiment analysis"
        
        research_plan = self.plan_research(request)
        print(f"\\nResearch Plan Created ({len(research_plan)} steps)")
        
        # Step 2: Retrieve past experience
        past_experiences = self.retrieve_relevant_experience(request)
        if past_experiences:
            print(f"Retrieved {len(past_experiences)} relevant past experiences")
        
        # Step 3: Execute Routing Workflow (Specialist Analysis)
        print(f"\\nExecuting Routing Workflow...")
        routing_results = self.routing_coordinator.route_analysis(request, symbol)
        
        # Step 4: Execute Prompt Chaining (News Processing) - included in news specialist
        print(f"\\nNews Processing Chain completed within specialist analysis")
        
        # Step 5: Execute Evaluator-Optimizer Workflow
        print(f"\\nExecuting Evaluator-Optimizer Workflow...")
        specialist_analyses = routing_results.get('specialist_analyses', {})
        optimization_results = self.evaluator_optimizer.optimize_analysis(symbol, specialist_analyses)
        
        # Step 6: Self-reflect on overall process
        print(f"\\nConducting self-reflection...")
        final_analysis = optimization_results.get('final_analysis', '')
        reflection = self.self_reflect(final_analysis)
        
        # Step 7: Learn from this experience
        print(f"\\nLearning from this research experience...")
        self.learn_from_experience(request, final_analysis, reflection)
        
        # Compile comprehensive results
        comprehensive_results = {
            'symbol': symbol,
            'request': request,
            'research_plan': research_plan,
            'past_experiences': past_experiences,
            'routing_results': routing_results,
            'optimization_results': optimization_results,
            'self_reflection': reflection,
            'execution_log': self.execution_log,
            'timestamp': datetime.now().isoformat(),
            'agent_name': self.name
        }
        
        print(f"\\nComprehensive research completed!")
        print(f"Final Analysis Quality Score: {reflection.get('overall_score', 'N/A')}/10")
        
        return comprehensive_results
    
    def generate_investment_report(self, research_results: Dict[str, Any]) -> str:
        """Generate a formatted investment report from research results"""
        symbol = research_results.get('symbol', 'N/A')
        final_analysis = research_results.get('optimization_results', {}).get('final_analysis', '')
        reflection = research_results.get('self_reflection', {})
        
        report_template = f"""
        # Investment Research Report: {symbol}
        **Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        **Research Quality Score:** {reflection.get('overall_score', 'N/A')}/10

        ## Executive Summary
        {final_analysis[:500]}...

        ## Key Research Insights
        - **Research Plan Execution:** {len(research_results.get('research_plan', []))} planned steps completed
        - **Specialist Analyses:** {len(research_results.get('routing_results', {}).get('specialist_analyses', {}))} specialist reports generated
        - **Analysis Iterations:** {len(research_results.get('optimization_results', {}).get('optimization_iterations', []))} optimization cycles
        - **Quality Improvements:** {'Yes' if research_results.get('optimization_results', {}).get('improvement_achieved', False) else 'No'}

        ## Analysis Quality Assessment
        - **Strengths:** {', '.join(reflection.get('strengths', ['Analysis completed']))}
        - **Recommendations:** {', '.join(reflection.get('recommendations', ['Continue monitoring']))}

        ## Full Analysis
        {final_analysis}

        ---
        *This report was generated by an AI Investment Research Agent.*
        """
        
        return report_template

# Initialize main research agent
main_research_agent = MainInvestmentResearchAgent(agent_memory)
print("Main Investment Research Agent Coordinator initialized!")

Main Investment Research Agent Coordinator initialized!


## 9. Visualization and Reporting Tools

In [27]:
class InvestmentVisualizer:
    """Create visualizations for investment research results"""
    
    def __init__(self):
        plt.style.use('default')
        sns.set_theme()
    
    def create_stock_price_chart(self, symbol: str, period: str = "6mo"):
        """Create stock price chart with volume"""
        try:
            stock = yf.Ticker(symbol)
            hist = stock.history(period=period)
            
            if hist.empty:
                return None
            
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), 
                                          gridspec_kw={'height_ratios': [3, 1]})
            
            # Price chart
            ax1.plot(hist.index, hist['Close'], linewidth=2, color='blue', label='Close Price')
            ax1.fill_between(hist.index, hist['Low'], hist['High'], alpha=0.3, color='lightblue')
            ax1.set_title(f'{symbol} Stock Price - {period.upper()}', fontsize=16, fontweight='bold')
            ax1.set_ylabel('Price ($)', fontsize=12)
            ax1.legend()
            ax1.grid(True, alpha=0.3)
            
            # Volume chart
            ax2.bar(hist.index, hist['Volume'], alpha=0.7, color='orange')
            ax2.set_title('Trading Volume', fontsize=12)
            ax2.set_ylabel('Volume', fontsize=10)
            ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            return fig
        
        except Exception as e:
            print(f"Error creating price chart: {e}")
            return None
    
    def create_sentiment_analysis_chart(self, news_analysis: Dict[str, Any]):
        """Create sentiment analysis visualization"""
        try:
            insights = news_analysis.get('insights', {})
            sentiment_dist = insights.get('sentiment_distribution', {})
            
            if not sentiment_dist:
                return None
            
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
            
            # Sentiment pie chart
            labels = list(sentiment_dist.keys())
            sizes = list(sentiment_dist.values())
            colors = ['green', 'red', 'gray']
            
            ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
            ax1.set_title('News Sentiment Distribution', fontsize=14, fontweight='bold')
            
            # Key themes bar chart
            themes = insights.get('key_themes', [])[:5]  # Top 5 themes
            if themes:
                ax2.barh(range(len(themes)), [1] * len(themes), color='steelblue')
                ax2.set_yticks(range(len(themes)))
                ax2.set_yticklabels(themes)
                ax2.set_title('Key News Themes', fontsize=14, fontweight='bold')
                ax2.set_xlabel('Frequency')
            
            plt.tight_layout()
            return fig
        
        except Exception as e:
            print(f"Error creating sentiment chart: {e}")
            return None
    
    def create_analysis_quality_dashboard(self, optimization_results: Dict[str, Any]):
        """Create analysis quality improvement dashboard"""
        try:
            iterations = optimization_results.get('optimization_iterations', [])
            if not iterations:
                return None
            
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
            
            # Quality score progression
            scores = [iter_data['evaluation'].get('overall_score', 0) for iter_data in iterations]
            ax1.plot(range(1, len(scores) + 1), scores, marker='o', linewidth=2, markersize=8)
            ax1.set_title('Analysis Quality Score Progression', fontsize=14, fontweight='bold')
            ax1.set_xlabel('Iteration')
            ax1.set_ylabel('Quality Score (1-10)')
            ax1.grid(True, alpha=0.3)
            ax1.set_ylim(0, 10)
            
            # Final evaluation scores
            if iterations:
                final_eval = iterations[-1]['evaluation']
                categories = list(final_eval.get('scores', {}).keys())
                values = list(final_eval.get('scores', {}).values())
                
                bars = ax2.bar(range(len(categories)), values, color='steelblue')
                ax2.set_title('Final Analysis Quality Breakdown', fontsize=14, fontweight='bold')
                ax2.set_ylabel('Score (1-10)')
                ax2.set_xticks(range(len(categories)))
                ax2.set_xticklabels(categories, rotation=45, ha='right')
                ax2.set_ylim(0, 10)
                
                # Add value labels on bars
                for bar, value in zip(bars, values):
                    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                            f'{value:.1f}', ha='center', va='bottom')
            
            # Grade distribution (if multiple analyses)
            ax3.text(0.5, 0.5, f"Final Grade: {iterations[-1]['evaluation'].get('grade', 'N/A')}", 
                    ha='center', va='center', fontsize=24, fontweight='bold',
                    transform=ax3.transAxes)
            ax3.set_title('Analysis Grade', fontsize=14, fontweight='bold')
            ax3.axis('off')
            
            # Improvement areas
            if iterations:
                improvements = iterations[-1]['evaluation'].get('specific_improvements', [])[:5]
                if improvements:
                    ax4.barh(range(len(improvements)), [1] * len(improvements), color='orange')
                    ax4.set_yticks(range(len(improvements)))
                    ax4.set_yticklabels([imp[:30] + '...' if len(imp) > 30 else imp for imp in improvements])
                    ax4.set_title('Key Improvement Areas', fontsize=14, fontweight='bold')
                    ax4.set_xlabel('Priority')
            
            plt.tight_layout()
            return fig
        
        except Exception as e:
            print(f"Error creating quality dashboard: {e}")
            return None
    
    def create_comprehensive_dashboard(self, research_results: Dict[str, Any]):
        """Create comprehensive research dashboard"""
        symbol = research_results.get('symbol', 'N/A')
        
        # Create individual charts
        price_chart = self.create_stock_price_chart(symbol)
        
        # Get news analysis from routing results
        routing_results = research_results.get('routing_results', {})
        specialist_analyses = routing_results.get('specialist_analyses', {})
        news_analysis = specialist_analyses.get('news', {}).get('raw_news_analysis', {})
        sentiment_chart = self.create_sentiment_analysis_chart(news_analysis)
        
        optimization_results = research_results.get('optimization_results', {})
        quality_dashboard = self.create_analysis_quality_dashboard(optimization_results)
        
        return {
            'price_chart': price_chart,
            'sentiment_chart': sentiment_chart,
            'quality_dashboard': quality_dashboard
        }

# Initialize visualizer
visualizer = InvestmentVisualizer()
print("Investment Visualizer created!")

Investment Visualizer created!


## 10. Gradio Web Interface

In [28]:
def analyze_stock(symbol, analysis_type="Comprehensive Analysis", include_visualizations=True):
    """Main function for Gradio interface"""
    try:
        # Input validation
        if not symbol or len(symbol.strip()) == 0:
            return "ERROR: Please enter a valid stock symbol", None, None, None, None
        
        symbol = symbol.upper().strip()
        
        # Create analysis request
        request_mapping = {
            "Comprehensive Analysis": f"Conduct comprehensive investment analysis for {symbol} including technical, fundamental, and sentiment analysis",
            "Technical Analysis Only": f"Perform detailed technical analysis for {symbol} focusing on price trends, indicators, and chart patterns", 
            "Fundamental Analysis Only": f"Conduct fundamental analysis for {symbol} focusing on financials, valuation, and business prospects",
            "News & Sentiment Only": f"Analyze recent news and market sentiment for {symbol}",
            "Quick Overview": f"Provide a quick investment overview for {symbol}"
        }
        
        request = request_mapping.get(analysis_type, request_mapping["Comprehensive Analysis"])
        
        # Perform analysis
        print(f"Starting {analysis_type} for {symbol}...")
        research_results = main_research_agent.conduct_comprehensive_research(symbol, request)
        
        # Generate report
        report = main_research_agent.generate_investment_report(research_results)
        
        # Create visualizations if requested
        price_chart = None
        sentiment_chart = None
        quality_chart = None
        
        if include_visualizations:
            try:
                dashboard = visualizer.create_comprehensive_dashboard(research_results)
                price_chart = dashboard.get('price_chart')
                sentiment_chart = dashboard.get('sentiment_chart') 
                quality_chart = dashboard.get('quality_dashboard')
            except Exception as e:
                print(f"Visualization error: {e}")
        
        # Prepare summary stats
        reflection = research_results.get('self_reflection', {})
        optimization_results = research_results.get('optimization_results', {})
        
        summary_stats = f"""
        **Analysis Summary:**
        - Quality Score: {reflection.get('overall_score', 'N/A')}/10
        - Analysis Grade: {optimization_results.get('final_evaluation', {}).get('grade', 'N/A')}
        - Optimization Iterations: {len(optimization_results.get('optimization_iterations', []))}
        - Specialists Consulted: {len(research_results.get('routing_results', {}).get('specialist_analyses', {}))}
        - Research Plan Steps: {len(research_results.get('research_plan', []))}

        **Workflow Patterns Executed:**
        - Prompt Chaining (News Processing Pipeline)
        - Routing (Specialist Agent Coordination) 
        - Evaluator-Optimizer (Quality Refinement Loop)

        **Agent Capabilities Demonstrated:**
        - Dynamic Planning & Tool Usage
        - Self-Reflection & Quality Assessment  
        - Learning & Memory Across Sessions
        - Multi-Source Data Integration
        """
        
        return report, summary_stats, price_chart, sentiment_chart, quality_chart
        
    except Exception as e:
        error_msg = f"Analysis Error: {str(e)}\n\nPlease check the stock symbol and try again."
        return error_msg, None, None, None, None

# Create Gradio interface
def create_gradio_interface():
    """Create and configure the Gradio web interface"""
    
    with gr.Blocks(title="AI Investment Research Agent", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # AI Investment Research Agent - Multi-Agent System
        
        **Advanced Investment Analysis using Three Workflow Patterns:**
        
        **Prompt Chaining**: News → Preprocess → Classify → Extract → Summarize  
        **Routing**: Specialist Agents (Technical, Fundamental, News Analysis)  
        **Evaluator-Optimizer**: Generate → Evaluate → Refine Analysis
        
        **Powered by:** Azure OpenAI, LangChain, Multi-Source Data Integration
        """)
        
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("## Analysis Configuration")
                
                symbol_input = gr.Textbox(
                    label="Stock Symbol",
                    placeholder="Enter stock symbol (e.g., AAPL, MSFT, TSLA)",
                    value="AAPL"
                )
                
                analysis_type = gr.Dropdown(
                    choices=[
                        "Comprehensive Analysis",
                        "Technical Analysis Only", 
                        "Fundamental Analysis Only",
                        "News & Sentiment Only",
                        "Quick Overview"
                    ],
                    value="Comprehensive Analysis",
                    label="Analysis Type"
                )
                
                include_viz = gr.Checkbox(
                    label="Include Visualizations",
                    value=True
                )
                
                analyze_btn = gr.Button(
                    "Start Analysis", 
                    variant="primary",
                    size="lg"
                )
                
                gr.Markdown("""
                ### System Features:
                - **Agent Functions**: Planning, Tool Usage, Self-Reflection, Learning
                - **Data Sources**: Yahoo Finance, NewsAPI, FRED, Alpha Vantage
                - **Memory System**: FAISS Vector Database
                - **Quality Assurance**: Automated evaluation and refinement
                """)
            
            with gr.Column(scale=2):
                gr.Markdown("## Analysis Results")
                
                summary_output = gr.Markdown(label="Analysis Summary")
                
                with gr.Tabs():
                    with gr.TabItem("Investment Report"):
                        report_output = gr.Markdown(
                            label="Investment Analysis Report",
                            value="Run analysis to see results..."
                        )
                    
                    with gr.TabItem("Price Chart"):
                        price_plot = gr.Plot(label="Stock Price Analysis")
                    
                    with gr.TabItem("Sentiment Analysis"):
                        sentiment_plot = gr.Plot(label="News Sentiment Analysis")
                    
                    with gr.TabItem("Quality Dashboard"):
                        quality_plot = gr.Plot(label="Analysis Quality Metrics")
        
        # Event handlers
        analyze_btn.click(
            fn=analyze_stock,
            inputs=[symbol_input, analysis_type, include_viz],
            outputs=[report_output, summary_output, price_plot, sentiment_plot, quality_plot]
        )
        
        # Example inputs
        gr.Markdown("""
        ### Try These Examples:
        - **AAPL**: Apple Inc. - Tech giant with strong fundamentals
        - **TSLA**: Tesla Inc. - High volatility growth stock  
        - **MSFT**: Microsoft Corp. - Stable large-cap technology
        - **NVDA**: NVIDIA Corp. - AI and semiconductor leader
        """)
    
    return demo

# Create the interface
gradio_demo = create_gradio_interface()
print("Gradio interface created! Ready to launch.")

Gradio interface created! Ready to launch.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


## 11. Testing and Demo

In [29]:
# Test the system with a sample analysis
def test_system():
    """Test the investment research agent system"""
    print("Testing Investment Research Agent System")
    print("=" * 50)
    
    try:
        # Test 1: Basic tool functionality
        print("\\n1. Testing data source tools...")
        stock_data = get_stock_data.invoke({"symbol": "AAPL", "period": "1mo"})
        print(f"Stock data tool working: {len(stock_data)} characters retrieved")
        
        # Test 2: Memory system
        print("\\n2. Testing memory system...")
        test_memory_content = "Test memory entry for system validation"
        agent_memory.add_memory(test_memory_content, {"type": "test", "timestamp": datetime.now().isoformat()})
        memories = agent_memory.search_memory("test", k=1)
        print(f"Memory system working: {len(memories)} memories retrieved")
        
        # Test 3: News processing chain
        print("\\n3. Testing news processing chain...")
        try:
            news_result = news_chain.ingest_news("AAPL")
            print(f"News processing working: {len(news_result)} articles processed")
        except Exception as e:
            print(f"News processing issue: {e}")
        
        # Test 4: Agent coordination
        print("\\n4. Testing main research agent...")
        # This is a lightweight test - full test would take longer
        plan = main_research_agent.plan_research("Quick test analysis for AAPL")
        print(f"Research planning working: {len(plan)} steps planned")
        
        print("\\nSystem tests completed successfully!")
        print("Ready for full analysis demonstration")
        
    except Exception as e:
        print(f"System test error: {e}")
        return False
    
    return True

# Run system tests
test_success = test_system()

Testing Investment Research Agent System
\n1. Testing data source tools...


INFO:httpx:HTTP Request: POST https://sasw-mgfv7ds1-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://sasw-mgfv7ds1-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://sasw-mgfv7ds1-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Stock data tool working: 81 characters retrieved
\n2. Testing memory system...
Memory system working: 1 memories retrieved
\n3. Testing news processing chain...
News API error: Error fetching news for AAPL: HTTPSConnectionPool(host='newsapi.org', port=443): Max retries exceeded with url: /v2/everything?q=AAPL&from=2025-10-05&language=en&sortBy=relevancy&pageSize=10 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)')))
News processing working: 0 articles processed
\n4. Testing main research agent...
News API error: Error fetching news for AAPL: HTTPSConnectionPool(host='newsapi.org', port=443): Max retries exceeded with url: /v2/everything?q=AAPL&from=2025-10-05&language=en&sortBy=relevancy&pageSize=10 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)')))
News proces

INFO:httpx:HTTP Request: POST https://sasw-mgfv7ds1-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-5-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Research planning working: 37 steps planned
\nSystem tests completed successfully!
Ready for full analysis demonstration


In [30]:
# Launch the Gradio interface
if test_success:
    
    # Launch the interface
    # gradio_demo.launch(share=True, debug=True)
    
    # For notebook environment, use queue
    gradio_demo.queue().launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=False,
        debug=True,
        show_error=True
    )
else:
    print("System tests failed. Please check the configuration and try again.")

* Running on local URL:  http://0.0.0.0:7860


INFO:httpx:HTTP Request: GET http://localhost:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://localhost:7860/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://localhost:7860/ "HTTP/1.1 200 OK"


* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
